# "Vanilla" YOLO modeļa spēju novērtēšana malārijas patogēnu noteikšanai

Šajā notebook'ā mēs:

- izmantojam **gatavu YOLO modeli** (piemēram, `yolov8n.pt`, trenētu uz COCO),
- palaižam to uz malārijas datu kopas attēliem,
- vizuāli salīdzinām, ko modelis detektē, ar aktualajām anotācijām,
- saprotam, kāpēc nepieciešama **piemācīšana uz konkrēto medicīnas datu kopu**.

In [ ]:
# Ja ultralytics nav instalēts, skolēni var palaist:
# !pip install ultralytics 

from ultralytics import YOLO    # Bibliotēka, kura piedāvā YOLO modeli
import numpy as np
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

YOLO_ROOT = Path("data/malaria_yolo")
IMAGES_VAL = YOLO_ROOT / "images" / "val"
LABELS_VAL = YOLO_ROOT / "labels" / "val"

# Tie paši klašu nosaukumi, ko norādījam datu sagatavošanas laikā
infected_classes = [
    "trophozoite",
    "ring",
    "schizont",
    "gametocyte",
]

In [ ]:
# Šis ir YOLOv8 nano modelis, trenēts uz COCO datu kopu (nevis uz malāriju).
model = YOLO("yolov8n.pt")

In [ ]:
def load_yolo_labels(label_path: Path):
    """
    Nolasa YOLO formāta .txt anotāciju:
    katra rinda: class_id xc yc w h (visi [0..1]).
    Atgriež sarakstu ar (class_id, xc, yc, w, h).
    """
    boxes = []
    if not label_path.exists():
        return boxes
    with open(label_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            cls_id = int(parts[0])
            xc, yc, w, h = map(float, parts[1:])
            boxes.append((cls_id, xc, yc, w, h))
    return boxes

In [ ]:
def yolo_to_xyxy(xc, yc, w, h, img_w, img_h):
    """Konvertē YOLO normalizētos koordinates uz pikseļiem (x1,y1,x2,y2)."""
    cx = xc * img_w
    cy = yc * img_h
    bw = w * img_w
    bh = h * img_h
    x1 = cx - bw / 2
    y1 = cy - bh / 2
    x2 = cx + bw / 2
    y2 = cy + bh / 2
    return x1, y1, x2, y2


def draw_boxes(image: Image.Image, boxes, color, labels=None):
    """
    boxes: saraksts ar (x1,y1,x2,y2)
    labels: saraksts ar teksta anotācijām (vai None)
    """
    img = image.copy()
    draw = ImageDraw.Draw(img)
    for i, (x1, y1, x2, y2) in enumerate(boxes):
        draw.rectangle([x1, y1, x2, y2], outline=color, width=8)
        if labels is not None:
            draw.text((x1, y1), labels[i], fill=color)
    return img

In [ ]:
val_images = sorted(list(IMAGES_VAL.glob("*.jpg")))[:5]  # paņemam 5 piemērus (var mainīt)

len(val_images), val_images[:3]

In [ ]:
for img_path in val_images:
    print("Attēls:", img_path.name)
    image = Image.open(img_path).convert("RGB")
    w, h = image.size

    # Ground truth (malārijas patogēni)
    label_path = LABELS_VAL / (img_path.stem + ".txt")
    gt_boxes_yolo = load_yolo_labels(label_path)

    gt_xyxy = []
    gt_labels = []
    for (cls_id, xc, yc, bw, bh) in gt_boxes_yolo:
        x1, y1, x2, y2 = yolo_to_xyxy(xc, yc, bw, bh, w, h)
        gt_xyxy.append((x1, y1, x2, y2))
        gt_labels.append(infected_classes[cls_id])

    # Vanilla YOLO (COCO) prognozes
    results = model(img_path)  # noklusēti: imgsz=640
    pred = results[0]

    pred_xyxy = []
    pred_labels = []
    if pred.boxes is not None and len(pred.boxes) > 0:
        # YOLOv8 dod xyxy normalizētos koordinātes (xyxyn) vai pikseļos (xyxy)
        # Šeit izmantojam xyxy pikseļos
        boxes_xyxy = pred.boxes.xyxy.cpu().numpy()
        conf = pred.boxes.conf.cpu().numpy()
        cls_ids = pred.boxes.cls.cpu().numpy().astype(int)

        for (bx1, by1, bx2, by2), c, cid in zip(boxes_xyxy, conf, cls_ids):
            pred_xyxy.append((bx1, by1, bx2, by2))
            pred_labels.append(f"{model.names[cid]} {c:.2f}")

    # Zīmējam
    img_gt = draw_boxes(image, gt_xyxy, color="green", labels=gt_labels)
    img_pred = draw_boxes(image, pred_xyxy, color="red", labels=pred_labels)

    fig, axs = plt.subplots(1, 3, figsize=(20, 10))
    axs[0].imshow(image)
    axs[0].set_title("Oriģinālais attēls")
    axs[0].axis("off")

    axs[1].imshow(img_gt)
    axs[1].set_title("Ground truth (malārijas patogēni, zaļš)")
    axs[1].axis("off")

    axs[2].imshow(img_pred)
    axs[2].set_title("Vanilla YOLO (COCO) prognozes, sarkans")
    axs[2].axis("off")

    plt.tight_layout()
    plt.show()